Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp
from htsworkflow.util.api import (
    add_auth_options,
    make_auth_from_opts,
    HtswApi,
)

In [4]:
config = configparser.ConfigParser()
config.read([os.path.expanduser('~/.htsworkflow.ini'),
             '/etc/htsworkflow.ini'
             ])

SECTION = 'sequence_archive'
if config.has_section(SECTION):
    apiid = config.get(SECTION, 'apiid')
    apikey = config.get(SECTION, 'apikey')
    apihost = config.get(SECTION, 'host')

auth = {'apiid': apiid, 'apikey': apikey }
htsw = HtswApi(apihost, auth)

In [5]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-22951-23100-rush-brain.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [6]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [7]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
        biosample.loc[i, 'source'] = obj['source']['@id']
            
biosample

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,description,library_id:skip,cDNA_sample:skip,donor,organism,source,lab,award
0,NaN,NaN,barbara-wold:ENC4_cDNA319,/biosample-types/tissue_UBERON_0006483/,mid frontal cortex; dorsolateral prefrontal co...,RUSH_AD_MFC_E5115600_ENC4_cDNA319,22951,ENC4_cDNA319,john-stamatoyannopoulos:E5115600,/organisms/human/,/sources/rush-university/,barbara-wold,UM1HG009443
1,NaN,NaN,barbara-wold:ENC4_cDNA320,/biosample-types/tissue_UBERON_0006483/,NaN,RUSH_AD_MFC_E6260868_ENC4_cDNA320,22952,ENC4_cDNA320,john-stamatoyannopoulos:E6260868,/organisms/human/,/sources/rush-university/,barbara-wold,UM1HG009443
2,NaN,NaN,barbara-wold:ENC4_cDNA321,/biosample-types/tissue_UBERON_0006483/,NaN,RUSH_AD_MFC_E2585111_ENC4_cDNA321,22953,ENC4_cDNA321,john-stamatoyannopoulos:E2585111,/organisms/human/,/sources/rush-university/,barbara-wold,UM1HG009443
3,NaN,NaN,barbara-wold:ENC4_cDNA322,/biosample-types/tissue_UBERON_0006483/,NaN,RUSH_AD_MFC_E7431498_ENC4_cDNA322,22954,ENC4_cDNA322,john-stamatoyannopoulos:E7431498,/organisms/human/,/sources/rush-university/,barbara-wold,UM1HG009443
4,NaN,NaN,barbara-wold:ENC4_cDNA323,/biosample-types/tissue_UBERON_0006483/,NaN,RUSH_AD_MFC_E7286104_ENC4_cDNA323,22955,ENC4_cDNA323,john-stamatoyannopoulos:E7286104,/organisms/human/,/sources/rush-university/,barbara-wold,UM1HG009443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,NaN,NaN,barbara-wold:ENC4_cDNA433,/biosample-types/tissue_UBERON_0006483/,NaN,RUSH_AD_MFC_E8505753_ENC4_cDNA433,23096,ENC4_cDNA433,john-stamatoyannopoulos:E8505753,/organisms/human/,/sources/rush-university/,barbara-wold,UM1HG009443
115,NaN,NaN,barbara-wold:ENC4_cDNA434,/biosample-types/tissue_UBERON_0006483/,NaN,RUSH_AD_MFC_E8392249_ENC4_cDNA434,23097,ENC4_cDNA434,john-stamatoyannopoulos:E8392249,/organisms/human/,/sources/rush-university/,barbara-wold,UM1HG009443
116,NaN,NaN,barbara-wold:ENC4_cDNA435,/biosample-types/tissue_UBERON_0006483/,NaN,RUSH_AD_MFC_E8895621_ENC4_cDNA435,23098,ENC4_cDNA435,john-stamatoyannopoulos:E8895621,/organisms/human/,/sources/rush-university/,barbara-wold,UM1HG009443
117,NaN,NaN,barbara-wold:ENC4_cDNA436,/biosample-types/tissue_UBERON_0006483/,NaN,RUSH_AD_MFC_E1652886_ENC4_cDNA436,23099,ENC4_cDNA436,john-stamatoyannopoulos:E1652886,/organisms/human/,/sources/rush-university/,barbara-wold,UM1HG009443


In [8]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Biosamples

In [22]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [21]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [26]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/', 
                            libraries,
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/stranded-22951-23100-rush-brain.xlsx
0


In [25]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [29]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [28]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [30]:
print(server.server)
print(spreadsheet_name)
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True, 
                            dry_run=False, 
                            validator=validator)
print(len(created))

www.encodeproject.org
/home/diane/woldlab/ENCODE/stranded-22951-23100-rush-brain.xlsx
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:03:51.796249+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:22951_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR098AHH/', 'library': '/libraries/ENCLB245ZCF/', 'status': 'in progress', '@id': '/replicates/e8d63970-156e-42d2-a1b6-d8141093aaad/', '@type': ['Replicate', 'Item'], 'uuid': 'e8d63970-156e-42d2-a1b6-d8141093aaad'}]}
row 0 created: e8d63970-156e-42d2-a1b6-d8141093aaad
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:03:52.075586+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:22952_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:03:55.492577+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:22965_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR033DKJ/', 'library': '/libraries/ENCLB807IWP/', 'status': 'in progress', '@id': '/replicates/341586a1-ba75-4011-9a40-32904cfa3e8c/', '@type': ['Replicate', 'Item'], 'uuid': '341586a1-ba75-4011-9a40-32904cfa3e8c'}]}
row 14 created: 341586a1-ba75-4011-9a40-32904cfa3e8c
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:03:55.783413+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:22966_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR274IOR/', 'library': '/libraries/ENCLB389ULX/'

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:03:59.324009+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:22979_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR057PZA/', 'library': '/libraries/ENCLB273FTJ/', 'status': 'in progress', '@id': '/replicates/ab0a8f45-18c2-4338-847d-d0a36b22cdfc/', '@type': ['Replicate', 'Item'], 'uuid': 'ab0a8f45-18c2-4338-847d-d0a36b22cdfc'}]}
row 28 created: ab0a8f45-18c2-4338-847d-d0a36b22cdfc
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:03:59.595417+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:22980_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR910XQY/', 'library': '/libraries/ENCLB334ZTD/'

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:03.177927+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:22993_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR105NCS/', 'library': '/libraries/ENCLB860BOM/', 'status': 'in progress', '@id': '/replicates/daa6aa29-7d2c-4237-9383-6fc929316d94/', '@type': ['Replicate', 'Item'], 'uuid': 'daa6aa29-7d2c-4237-9383-6fc929316d94'}]}
row 42 created: daa6aa29-7d2c-4237-9383-6fc929316d94
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:03.428588+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:22994_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR192DXS/', 'library': '/libraries/ENCLB027MFO/'

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:06.933357+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23038_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR040FYZ/', 'library': '/libraries/ENCLB547TOX/', 'status': 'in progress', '@id': '/replicates/e55745a5-121e-45b9-8c0c-54312a699346/', '@type': ['Replicate', 'Item'], 'uuid': 'e55745a5-121e-45b9-8c0c-54312a699346'}]}
row 56 created: e55745a5-121e-45b9-8c0c-54312a699346
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:07.212971+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23039_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR143RJN/', 'library': '/libraries/ENCLB245CZY/'

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:10.787240+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23052_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR691XPO/', 'library': '/libraries/ENCLB539BFQ/', 'status': 'in progress', '@id': '/replicates/27541572-3c3f-4272-b60a-40d908d326ee/', '@type': ['Replicate', 'Item'], 'uuid': '27541572-3c3f-4272-b60a-40d908d326ee'}]}
row 70 created: 27541572-3c3f-4272-b60a-40d908d326ee
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:11.060637+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23053_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR701ZXF/', 'library': '/libraries/ENCLB410GIT/'

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:14.588532+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23066_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR393ONB/', 'library': '/libraries/ENCLB594VVB/', 'status': 'in progress', '@id': '/replicates/152519bc-876f-4181-bb4d-573cfdff2559/', '@type': ['Replicate', 'Item'], 'uuid': '152519bc-876f-4181-bb4d-573cfdff2559'}]}
row 84 created: 152519bc-876f-4181-bb4d-573cfdff2559
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:14.861997+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23067_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR890CWQ/', 'library': '/libraries/ENCLB799OWB/'

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:18.412745+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23080_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR352TIK/', 'library': '/libraries/ENCLB289ZDX/', 'status': 'in progress', '@id': '/replicates/a42f1f68-2d17-4154-a409-c548c62ba865/', '@type': ['Replicate', 'Item'], 'uuid': 'a42f1f68-2d17-4154-a409-c548c62ba865'}]}
row 98 created: a42f1f68-2d17-4154-a409-c548c62ba865
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:18.691673+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23081_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR111VTZ/', 'library': '/libraries/ENCLB679DUT/'

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:22.392441+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23094_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR140RTO/', 'library': '/libraries/ENCLB745ZLH/', 'status': 'in progress', '@id': '/replicates/460ef87f-0546-445b-b4e1-2fb7eb06a46d/', '@type': ['Replicate', 'Item'], 'uuid': '460ef87f-0546-445b-b4e1-2fb7eb06a46d'}]}
row 112 created: 460ef87f-0546-445b-b4e1-2fb7eb06a46d
Reponse {'status': 'success', '@type': ['result'], '@graph': [{'date_created': '2021-04-10T00:04:22.670336+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'aliases': ['barbara-wold:23095_b1_t1'], 'schema_version': '9', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'experiment': '/experiments/ENCSR807MST/', 'library': '/libraries/ENCLB291DTZ/

In [31]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [19]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))

121
